In [13]:
# здесь только импорт
import pandas as pd
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
import numpy as np
import math
from urllib.request import urlopen
from bs4 import BeautifulSoup
import string
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_datareader.data as web
import copy
import time
import warnings
from datetime import datetime, timedelta
from multiprocessing import Process
from joblib import Parallel, delayed, wrap_non_picklable_objects
import multiprocessing
from tqdm import tqdm
from tqdm import tqdm_notebook
from joblib import parallel_backend
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
import operator
import csv
from fake_useragent import UserAgent
import requests, json
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import itertools
import re
from datetime import date, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [14]:
print ('Собираем список тикеров, торгуемых на СПб бирже')

dataset = pd.read_csv('ListingSecurityList.csv', sep = ';')
dataset = dataset[['s_RTS_code', 'e_full_name']]
tickers_only = dataset['s_RTS_code'].values

len(tickers_only)

Собираем список тикеров, торгуемых на СПб бирже


1720

In [15]:
def parsing_web_data(j):

    try:
        tickerSymbol = tickers_only[j]
        tickerData = yf.Ticker(tickerSymbol)
        ticker_info = tickerData.info
        marketCap = ticker_info['marketCap']
        P_E = ticker_info['forwardPE']
        price = ticker_info['currentPrice']
    except:
        marketCap = 0
        P_E = 0
        price = 0
    dataset.loc[j, 'marketCap'] = marketCap
    dataset.loc[j, 'price'] = price
    dataset.loc[j, 'P/E'] = P_E
    
    try:
        ROE = ticker_info['returnOnEquity']
    except:
        ROE = 0
    dataset.loc[j, 'ROE'] = ROE
    
    
    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[j] + '/' + tickers_only[j] + '/net-income')
        rg_table = table[1]
        rg_table = rg_table.iloc[:, 1].values
        rg_table = filter(lambda v: v==v, rg_table)
        rg_table = [s.replace('$', '') for s in rg_table]
        rg_table = [s.replace(',', '') for s in rg_table]
        x = np.asarray(rg_table, dtype=np.float64)
        x_rev = np.flip(x)
        x_rev = x_rev[-21:]

        ttm_list = list()
        for i in range(len(x_rev)-3):
            ttm = x_rev[i] + x_rev[i+1] + x_rev[i+2] + x_rev[i+3]
            ttm_list.append(ttm)

        X = np.linspace(1, len(ttm_list), num=len(ttm_list))
        X = X.reshape(-1, 1)
        
        reg = LinearRegression().fit(X, ttm_list)
        determination = round(reg.score(X, ttm_list), 2)
        
        if (ttm_list[-1] < 100) or (ttm_list[0] < 100):
            growth = 0
        else:
            growth = round(((ttm_list[-1] / ttm_list[0])**(1/((len(x_rev)-1)/4)) - 1) * 100, 2)
    
        
       
    except:
        growth = 0
        determination = 0
    dataset.loc[j, 'growth'] = growth
    dataset.loc[j, 'determination'] = determination
    
      
    
    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[j] + '/' + tickers_only[j] + '/net-profit-margin')
        net_margin_mean = 0
        for i in range(4):
            net_margin_mean = net_margin_mean + float(table[0].iloc[:, [3]].values[i][0][:-1])
        net_margin_mean = round(net_margin_mean / 4, 2)
    except:
        net_margin_mean = 0
    dataset.loc[j, 'net_margin_mean'] = net_margin_mean

In [16]:
%%time

warnings.filterwarnings('ignore')

jobs_count = multiprocessing.cpu_count()

print ('Собираем фундаментальные показатели компаний')

with parallel_backend('threading', n_jobs = jobs_count):
    Parallel()(delayed(parsing_web_data)(m) for m in tqdm_notebook(range(len(tickers_only))))

Собираем фундаментальные показатели компаний


  0%|          | 0/1720 [00:00<?, ?it/s]

Wall time: 26min 52s


In [17]:
dataset.to_excel('all.xlsx')

In [229]:
# отбираем стабильно растущие с 2016 года

dataset_cut = copy.deepcopy(dataset)
dataset_cut.loc[dataset_cut['ROE'] < 0.12, 'ROE'] = None
dataset_cut.loc[dataset_cut['marketCap'] < 10000000000, 'marketCap'] = None
dataset_cut.loc[dataset_cut['growth'] < 10, 'growth'] = None
dataset_cut.loc[dataset_cut['growth'] > 40, 'growth'] = 40
dataset_cut.loc[dataset_cut['determination'] < 0.5, 'determination'] = None
dataset_cut.loc[dataset_cut['net_margin_mean'] < 5, 'net_margin_mean'] = None
dataset_cut.loc[dataset_cut['net_margin_mean'] > 35, 'net_margin_mean'] = 35

dataset_cut = dataset_cut.dropna().reset_index(drop=True)

minimum = np.min(dataset_cut.marketCap.values)
maximum = np.max(dataset_cut.marketCap.values)
dataset_cut['marketCap_score'] = dataset_cut['marketCap'].map(lambda x: (x - minimum)/(maximum - minimum))

minimum = np.min(dataset_cut.growth.values)
maximum = np.max(dataset_cut.growth.values)
dataset_cut['growth_score'] = dataset_cut['growth'].map(lambda x: (x - minimum)/(maximum - minimum))

minimum = np.min(dataset_cut.determination.values)
maximum = np.max(dataset_cut.determination.values)
dataset_cut['determination_score'] = dataset_cut['determination'].map(lambda x: (x - minimum)/(maximum - minimum))

minimum = np.min(dataset_cut.net_margin_mean.values)
maximum = np.max(dataset_cut.net_margin_mean.values)
dataset_cut['net_margin_mean_score'] = dataset_cut['net_margin_mean'].map(lambda x: (x - minimum)/(maximum - minimum))

dataset_cut['Score'] = dataset_cut["marketCap_score"] + dataset_cut["growth_score"] +  dataset_cut["determination_score"] +  dataset_cut["net_margin_mean_score"]

yf.pdr_override()
data = web.get_data_yahoo(list(dataset_cut['s_RTS_code'].values), start=str((date.today() - timedelta(days=360)).strftime("%Y-%m-%d")), end=str(date.today().strftime("%Y-%m-%d")), auto_adjust = True)

for k in tqdm_notebook(range(len(dataset_cut['s_RTS_code'].values))):
    dens = np.histogram(data['Close'][data['Close'].columns[k]].values[~np.isnan(data['Close'][data['Close'].columns[k]].values)])[0]
    prices = np.histogram(data['Close'][data['Close'].columns[k]].values[~np.isnan(data['Close'][data['Close'].columns[k]].values)])[1]
    for p in range(len(dens)):
        if dens[len(dens) - 2 - p] > dens[len(dens) - 1 - p] and dens[len(dens) - 2 - p] > dens[len(dens) - 3 - p] and dens[len(dens) - 2 - p] > 1.2*dens[-2-level]:
            level = p
            break

    nice_price = prices[6]
    price_now = data['Close'][data['Close'].columns[k]].values[~np.isnan(data['Close'][data['Close'].columns[k]].values)][-1]
    overprice = round(((price_now - nice_price) / nice_price) * 100, 2)

    dataset_cut.loc[k, 'overprice'] = overprice

dataset_cut = dataset_cut.sort_values(by = 'Score', ascending = False).reset_index(drop=True)

dataset_cut.to_excel('growth.xlsx')

[*********************100%***********************]  107 of 107 completed


  0%|          | 0/107 [00:00<?, ?it/s]

In [304]:
for k in tqdm_notebook(range(len(dataset_cut['s_RTS_code'].values))):
    response = requests.get('https://finance.yahoo.com/quote/' + dataset_cut['s_RTS_code'][k] +'/analysis/', headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    string_soup = str(soup)

    pos1 = string_soup.find('Next 5 Years (per annum)')
    string_soup = string_soup[pos1:]
    pos2 = string_soup.find('%')
    string_soup = string_soup[(pos2-6):pos2]
    pos3 = string_soup.find('>')
    try:
        future_growth = float(string_soup[pos3+1:])
    except:
        future_growth = 0
    dataset_cut.loc[k, 'future_growth'] = future_growth

  0%|          | 0/107 [00:00<?, ?it/s]

In [ ]:
X = dataset_cut[['marketCap_score', 'growth_score', 'determination_score', 'net_margin_mean_score']].values
y = dataset_cut[['P/E']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train, y_train.ravel())

predictions = rf.predict(X)

df_rf = pd.DataFrame({'Name': dataset_cut['s_RTS_code'].values,'Score': dataset_cut['Score'].values, 'Actual': y.ravel(), 'Predicted': predictions.ravel()})

df_rf['overprice_ML'] = round(100*(df_rf['Actual'] - df_rf['Predicted'])/df_rf['Predicted'], 2)

dataset_cut['overprice_ML'] = df_rf['overprice_ML']

In [316]:
# найдём RSI для growth
yf.pdr_override()
data = web.get_data_yahoo(list(dataset_cut['s_RTS_code'].values), start=str((date.today() - timedelta(days=100)).strftime("%Y-%m-%d")), end=str(date.today().strftime("%Y-%m-%d")), auto_adjust = True)

RSI_list = list()

for ticker in dataset_cut['s_RTS_code'].values:
    change_list = list()
    AvgU = 0
    AvgD = 0
    for i in range(16):
        change = (data['Close'][ticker][len(data['Close'])-i-1] - data['Close'][ticker][len(data['Close'])-2-i])/data['Close'][ticker][len(data['Close'])-2-i]
        change_list.append(change)

        
        if change > 0:
            AvgU = AvgU + change_list[i]
        if change < 0:
            AvgD = AvgD - change_list[i]
    RS = AvgU / AvgD
    RSI = 100 - 100/(1 + RS)
    RSI_list.append(RSI)
    
dataset_cut['RSI'] = RSI_list

[*********************100%***********************]  107 of 107 completed


In [20]:
def parsing_boost_data(j):

    try:
        tickerSymbol = tickers_only[j]
        tickerData = yf.Ticker(tickerSymbol)
        ticker_info = tickerData.info
        marketCap = ticker_info['marketCap']
        P_S = ticker_info['priceToSalesTrailing12Months']
        price = ticker_info['currentPrice']
    except:
        marketCap = 0
        P_S = 0
        price = 0
    dataset_new.loc[j, 'marketCap'] = marketCap
    dataset_new.loc[j, 'price'] = price
    dataset_new.loc[j, 'P/S'] = P_S
    
    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[j] + '/' + tickers_only[j] + '/gross-profit')
        rg_table = table[1]
        rg_table = rg_table.iloc[:, 1].values
        rg_table = filter(lambda v: v==v, rg_table)
        rg_table = [s.replace('$', '') for s in rg_table]
        rg_table = [s.replace(',', '') for s in rg_table]
        x = np.asarray(rg_table, dtype=np.float64)
        x_rev = np.flip(x)

        ttm_list = list()
        growth_list = list()

        if len(x_rev) <= 4:
            rg_table = table[0]
            rg_table = rg_table.iloc[:, 1].values
            rg_table = filter(lambda v: v==v, rg_table)
            rg_table = [s.replace('$', '') for s in rg_table]
            rg_table = [s.replace(',', '') for s in rg_table]
            x = np.asarray(rg_table, dtype=np.float64)
            x_rev = np.flip(x)
            for i in range(len(x_rev) - 1):
                growth_list.append((x_rev[i+1] - x_rev[i]) / x_rev[i])

            rev_growth = np.median(growth_list)

            if np.min(growth_list) < 0.1:
                rev_growth = 0

        else:
            for i in range(len(x_rev)-3):
                ttm = x_rev[i] + x_rev[i+1] + x_rev[i+2] + x_rev[i+3]
                ttm_list.append(ttm)


            for i in range(len(x_rev)-6):
                growth_list.append((ttm_list[i+3] - ttm_list[i])/ttm_list[i])

            rev_growth = np.median(growth_list[-8:])

            if np.min(growth_list[-8:]) < 0.15:
                rev_growth = 0
        
    except:
        rev_growth = 0
        
    dataset_new.loc[j, 'rev_growth'] = rev_growth
    
    
      
    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[j] + '/' + tickers_only[j] + '/net-income')
        rg_table = table[1]
        rg_table = rg_table.iloc[:, 1].values
        rg_table = filter(lambda v: v==v, rg_table)
        rg_table = [s.replace('$', '') for s in rg_table]
        rg_table = [s.replace(',', '') for s in rg_table]
        x = np.asarray(rg_table, dtype=np.float64)
        x_rev = np.flip(x)
        x_rev = x_rev[-12:]

        ttm_list = list()
        for i in range(len(x_rev)-3):
            ttm = x_rev[i] + x_rev[i+1] + x_rev[i+2] + x_rev[i+3]
            ttm_list.append(ttm)

        X = np.linspace(1, len(ttm_list), num=len(ttm_list))
        X = X.reshape(-1, 1)
        
        reg = LinearRegression().fit(X, ttm_list)
        growth = round((reg.predict(X)[-1] - reg.predict(X)[0]) / abs(reg.predict(X)[0]), 2)
         
    except:
        growth = 0
        
    dataset_new.loc[j, 'eps_growth'] = growth
   
    try:
        table=pd.read_html('https://www.macrotrends.net/stocks/charts/' + tickers_only[j] + '/' + tickers_only[j] + '/net-profit-margin')
        net_margin_mean = 0
        for i in range(4):
            net_margin_mean = net_margin_mean + float(table[0].iloc[:, [3]].values[i][0][:-1])
        net_margin_mean = round(net_margin_mean / 4, 2)
    except:
        net_margin_mean = 0
    dataset_new.loc[j, 'net_margin_mean'] = net_margin_mean    

In [21]:
%%time

warnings.filterwarnings('ignore')

jobs_count = multiprocessing.cpu_count()

print ('Собираем фундаментальные показатели компаний')

dataset_new = pd.read_csv('ListingSecurityList.csv', sep = ';')
dataset_new = dataset_new[['s_RTS_code', 'e_full_name']]

with parallel_backend('threading', n_jobs = jobs_count):
    Parallel()(delayed(parsing_boost_data)(m) for m in tqdm_notebook(range(len(tickers_only))))

Собираем фундаментальные показатели компаний


  0%|          | 0/1720 [00:00<?, ?it/s]

Wall time: 32min 53s


In [22]:
dataset_new.to_excel('all_new.xlsx')

In [206]:
dataset_new_cut = copy.deepcopy(dataset_new)
dataset_new_cut.loc[dataset_new_cut['rev_growth'] <= 0, 'rev_growth'] = None
dataset_new_cut.loc[dataset_new_cut['rev_growth'] > 1, 'rev_growth'] = 1
dataset_new_cut.loc[dataset_new_cut['eps_growth'] < 0, 'eps_growth'] = 0
dataset_new_cut.loc[dataset_new_cut['eps_growth'] > 2, 'eps_growth'] = 2
dataset_new_cut.loc[dataset_new_cut['net_margin_mean'] > 15, 'net_margin_mean'] = 15
dataset_new_cut.loc[dataset_new_cut['net_margin_mean'] < -10, 'net_margin_mean'] = -10

dataset_new_cut = dataset_new_cut.dropna().reset_index(drop=True)

minimum = np.min(dataset_new_cut.rev_growth.values)
maximum = np.max(dataset_new_cut.rev_growth.values)
dataset_new_cut['rev_growth_score'] = dataset_new_cut['rev_growth'].map(lambda x: (x - minimum)/(maximum - minimum))

minimum = np.min(dataset_new_cut.eps_growth.values)
maximum = np.max(dataset_new_cut.eps_growth.values)
dataset_new_cut['eps_growth_score'] = dataset_new_cut['eps_growth'].map(lambda x: (x - minimum)/(maximum - minimum))

minimum = np.min(dataset_new_cut.net_margin_mean.values)
maximum = np.max(dataset_new_cut.net_margin_mean.values)
dataset_new_cut['net_margin_score'] = dataset_new_cut['net_margin_mean'].map(lambda x: (x - minimum)/(maximum - minimum))

dataset_new_cut['Score'] = 1.5 * dataset_new_cut['rev_growth_score'] + 1.5 * dataset_new_cut['eps_growth_score'] + dataset_new_cut['net_margin_score']

yf.pdr_override()
data = web.get_data_yahoo(list(dataset_new_cut['s_RTS_code'].values), start=str((date.today() - timedelta(days=120)).strftime("%Y-%m-%d")), end=str(date.today().strftime("%Y-%m-%d")), auto_adjust = True)
sale_cluster_list = list()
for k in tqdm_notebook(range(len(dataset_new_cut['s_RTS_code'].values))):
    try:
        nice_price = np.histogram(data['Close'][data['Close'].columns[k]].values)[1][6]
        price_now = data['Close'][data['Close'].columns[k]].values[-1]
        overprice = round(((price_now - nice_price) / nice_price) * 100, 2)
        
    except:
        overprice = -100
    dataset_new_cut.loc[k, 'overprice'] = overprice

    
dataset_new_cut = dataset_new_cut.sort_values(by = 'Score', ascending = False).reset_index(drop=True)

dataset_new_cut.to_excel('boost.xlsx')

[*********************100%***********************]  125 of 125 completed


  0%|          | 0/125 [00:00<?, ?it/s]

In [344]:
for k in tqdm_notebook(range(len(dataset_new_cut['s_RTS_code'].values))):
    response = requests.get('https://finance.yahoo.com/quote/' + dataset_new_cut['s_RTS_code'][k] +'/analysis/', headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    string_soup = str(soup)

    pos1 = string_soup.find('Next 5 Years (per annum)')
    string_soup = string_soup[pos1:]
    pos2 = string_soup.find('%')
    string_soup = string_soup[(pos2-6):pos2]
    pos3 = string_soup.find('>')
    try:
        future_growth = float(string_soup[pos3+1:])
    except:
        try:
            string_soup = str(soup)
            pos1 = string_soup.find('Next Year')
            string_soup = string_soup[pos1+1:]
            pos1 = string_soup.find('Next Year')
            string_soup = string_soup[pos1+1:]
            pos1 = string_soup.find('Next Year')
            string_soup = string_soup[pos1+1:]
            pos1 = string_soup.find('Next Year')
            string_soup = string_soup[pos1+1:]
            pos1 = string_soup.find('Next Year')
            string_soup = string_soup[pos1+1:]
            pos2 = string_soup.find('%')
            string_soup = string_soup[(pos2-6):pos2]
            pos3 = string_soup.find('>')
            future_growth = float(string_soup[pos3+1:])

        except:
            future_growth = 0
    dataset_new_cut.loc[k, 'future_growth'] = future_growth
    
minimum = np.min(dataset_new_cut.future_growth.values)
maximum = np.max(dataset_new_cut.future_growth.values)
dataset_new_cut['future_growth_score'] = dataset_new_cut['future_growth'].map(lambda x: (x - minimum)/(maximum - minimum))

dataset_new_cut['Score'] = 1.5 * dataset_new_cut['rev_growth_score'] + 1.5 * dataset_new_cut['eps_growth_score'] + 1.5 * dataset_new_cut['future_growth_score'] + dataset_new_cut['net_margin_score']
dataset_new_cut = dataset_new_cut.sort_values(by = 'Score', ascending = False).reset_index(drop=True)

  0%|          | 0/125 [00:00<?, ?it/s]

In [345]:
X = dataset_new_cut[['rev_growth', 'future_growth']].values
y = dataset_new_cut[['P/S']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train, y_train.ravel())

predictions = rf.predict(X)

df_rf = pd.DataFrame({'Name': dataset_new_cut['s_RTS_code'].values,'Score': dataset_new_cut['Score'].values, 'Actual': y.ravel(), 'Predicted': predictions.ravel()})

df_rf['overprice_ML'] = 100*(df_rf['Actual'] - df_rf['Predicted'])/df_rf['Predicted']

dataset_new_cut['overprice_ML'] = df_rf['overprice_ML']

In [337]:
# найдём RSI для boost
yf.pdr_override()
data = web.get_data_yahoo(list(dataset_new_cut['s_RTS_code'].values), start=str((date.today() - timedelta(days=50)).strftime("%Y-%m-%d")), end=str(date.today().strftime("%Y-%m-%d")), auto_adjust = True)

RSI_list = list()

for ticker in dataset_new_cut['s_RTS_code'].values:
    change_list = list()
    AvgU = 0
    AvgD = 0
    for i in range(15):
        change = (data['Close'][ticker][len(data['Close'])-i-1] - data['Close'][ticker][len(data['Close'])-i-2])/data['Close'][ticker][len(data['Close'])-i-2]
        change_list.append(change)

        
        if change > 0:
            AvgU = AvgU + change_list[i]
        if change < 0:
            AvgD = AvgD - change_list[i]
    RS = AvgU / AvgD
    RSI = 100 - 100/(1 + RS)
    RSI_list.append(RSI)
    
dataset_new_cut['RSI'] = RSI_list

[*********************100%***********************]  125 of 125 completed


In [347]:
dataset_new_cut.to_excel('boost.xlsx')
dataset_cut.to_excel('growth.xlsx')